This notebook aims to study the vulnerabilities and how their fixes relate to backported updates. 

The data we relied on are subject to a non-disclosure agreement. That means we are not allowed to share these data, so you'll have to trust us ;)

In [1]:
import pandas
import numpy as np
import matplotlib
import seaborn

from IPython.display import display

%matplotlib inline

In [2]:
FIG_SIZE = (8, 3)
FIG_SIZE_WIDE = (8, 2.5)

ECOSYSTEMS = ['NPM', 'Rubygems']
DATE_RANGE = pandas.to_datetime('2015-01-01'), pandas.to_datetime('2020-01-01')
CENSOR_DATE = pandas.to_datetime('2020-01-12')

PALETTE = seaborn.color_palette()
PAL_REL = np.take(seaborn.color_palette('muted'), [3, 8, 2, 0], axis=0)
COLORS = {'NPM': PALETTE[1], 'Rubygems': PALETTE[3]}

matplotlib.rcParams['figure.figsize'] = FIG_SIZE
matplotlib.rcParams['legend.framealpha'] = 1
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

SEED = 12345
SAVEFIG = False

def _savefig(fig, name):
    import os
    fig.savefig(
        os.path.join('..', 'figures', '{}.pdf'.format(name)),
        bbox_inches='tight'
    )
    
savefig = _savefig if SAVEFIG else lambda x, y: None

# Dataset

In [3]:
df_vuln = (
    pandas.read_csv('../data-raw/vulnerabilities.csv.gz', index_col=0, infer_datetime_format=True, parse_dates=['published', 'disclosed'])
    .rename(columns={
        'Id': 'id',
        'vuln_name': 'vulnerability', 
        'base': 'ecosystem', 
        'cvssScore': 'score',
        'fixedIn': 'fix', 
        'affecting': 'affect',
    })
    .replace({'ecosystem': {'npm': 'NPM', 'RubyGems': 'Rubygems'}})
)

In [4]:
df_vuln.describe(datetime_is_numeric=True)

,published,disclosed,score
count,2874,2874,2874.000000
mean,2018-04-04 09:14:39.331941632,2017-05-16 05:40:12.526096128,7.045616
min,2013-03-07 00:00:00,2006-08-14 00:00:00,0.000000
25%,2017-02-13 00:00:00,2016-03-16 06:00:00,5.900000
50%,2018-02-26 00:00:00,2018-01-15 00:00:00,7.200000
75%,2019-07-15 00:00:00,2019-05-06 00:00:00,8.100000
max,2020-04-12 00:00:00,2020-12-13 00:00:00,10.000000
std,NaN,NaN,1.700786


In [5]:
df_vuln.head()

,id,package,published,disclosed,severity,vulnerability,ecosystem,score,fix,affect
0,SNYK-JS-MBACKDOOR-565090,m-backdoor,2020-04-12,2020-04-10,critical,Malicious Package,NPM,9.8,undefined,*
1,SNYK-JS-PAYPALADAPTIVE-565089,paypal-adaptive,2020-04-12,2020-04-12,medium,Prototype Pollution,NPM,4.2,undefined,*
2,SNYK-JS-GRUNTUTILPROPERTY-565088,grunt-util-property,2020-04-12,2020-04-12,medium,Prototype Pollution,NPM,4.0,undefined,*
3,SNYK-JS-ELECTRON-565052,electron,2020-04-10,2020-03-06,high,Out-of-bounds Read,NPM,7.3,8.2.0,<8.2.0
4,SNYK-JS-ELECTRON-565051,electron,2020-04-10,2020-04-02,high,Heap Overflow,NPM,8.8,8.2.1,<8.2.1


## Data selection

Not all vulnerabilities are of interest for our work. 
First, only the ones affecting one of the packages we considered are useful. Second, only the ones that are fixed are interesting to study (indeed, there is no hope to find a backported fix if there is no fix!). 

In [6]:
df_required = dict()
df_dependents = dict()

for ecosystem in ECOSYSTEMS:
    print('Loading', ecosystem)
    print('.. required packages')
    df_required[ecosystem] = (
        pandas.read_csv(
            '../data/{}-required.csv.gz'.format(ecosystem),
            parse_dates=['date'],
            infer_datetime_format=True,
        )
    )
    
    print('.. dependent packages')
    df_dependents[ecosystem] = (
        pandas.read_csv(
            '../data/{}-dependents.csv.gz'.format(ecosystem),
        )
    )
print('Merging...')

df_required = pandas.concat([v.assign(ecosystem=k) for k,v in df_required.items()])
df_dependents = pandas.concat([v.assign(ecosystem=k) for k,v in df_dependents.items()])

print('Done!')

Loading NPM
.. required packages
.. dependent packages
Loading Rubygems
.. required packages
.. dependent packages
Merging...
Done!


In [7]:
df_vuln = (
    df_vuln
    .merge(
        df_required[['ecosystem', 'package']]
        .drop_duplicates()
        .assign(required=True),
        how='left',
        on=['ecosystem', 'package'],
    )
    .fillna({'required': False})
    .assign(fixed=lambda d: ~d['fix'].isin(['undefined']))
)

In [8]:
(
    df_vuln
    .groupby(['ecosystem', 'required', 'fixed'])
    .agg({'vulnerability': 'count'})
)

vulnerability
ecosystem required fixed               
NPM       False    False           1022
                   True             505
          True     False             47
                   True             614
Rubygems  False    False            110
                   True             247
          True     False             14
                   True             315

We'll focus on the ones being "required" and "fixed". 

In [9]:
df_vuln = df_vuln.query('required and fixed')

In [10]:
(
    df_vuln
    .groupby(['ecosystem', 'severity'])
    .agg({'vulnerability': 'count', 'package': 'nunique'})
)

vulnerability  package
ecosystem severity                        
NPM       critical             31       24
          high                217      138
          low                  40       36
          medium              326      195
Rubygems  critical             10        9
          high                 86       39
          low                  15        9
          medium              204       75

In [11]:
df_vuln.sample(n=10, random_state=SEED)

,id,package,published,disclosed,severity,vulnerability,ecosystem,score,fix,affect,required,fixed
840,SNYK-JS-NUXTDEVALUE-174322,@nuxt/devalue,2019-04-16,2019-04-12,high,Cross-site Scripting (XSS),NPM,8.3,1.2.3,<1.2.3,True,True
74,SNYK-JS-DOJO-559224,dojo,2020-03-04,2020-03-04,medium,Prototype Pollution,NPM,4.2,1.11.10||1.12.8||1.13.7||1.14.6||1.15.3||1.16.2,<1.11.10||>=1.12.0 <1.12.8||>=1.13.0 <1.13.7||...,True,True
442,SNYK-JS-KNEX-471962,knex,2019-10-07,2019-10-07,critical,SQL Injection,NPM,9.8,0.19.5,<0.19.5,True,True
2108,SNYK-JS-ISMYJSONVALID-10079,is-my-json-valid,2016-01-18,2016-01-18,high,Regular Expression Denial of Service (ReDoS),NPM,7.5,2.12.4,<2.12.4,True,True
515,SNYK-JS-WEBTORRENT-460351,webtorrent,2019-08-28,2019-08-27,low,Cross-site Scripting (XSS),NPM,3.1,0.107.6,<0.107.6,True,True
1781,SNYK-JS-SWAGGERUI-10423,swagger-ui,2017-03-01,2016-08-31,medium,Cross-site Scripting (XSS),NPM,6.1,2.2.3,<2.2.3,True,True
2232,SNYK-RUBY-REDARROW-483027,red-arrow,2019-11-11,2019-11-08,medium,Use of Uninitialized Variable,Rubygems,4.8,0.15.1,">=0.14.0, <0.15.1",True,True
1566,SNYK-JS-CORDOVAANDROID-10650,cordova-android,2017-02-27,2014-08-03,low,Information Exposure,NPM,3.1,3.5.1,<3.5.1,True,True
2728,SNYK-RUBY-NETLDAP-20146,net-ldap,2016-09-21,2014-02-12,low,Information Exposure,Rubygems,2.9,0.6.0,< 0.6.0,True,True
2716,SNYK-RUBY-ACTIONPACK-20158,actionpack,2016-09-21,2014-05-05,medium,Directory Traversal,Rubygems,4.3,4.1.1||4.0.5||3.2.18,"< 4.1.1, >= 4.1||< 4.0.5, >= 3.3||< 3.2.18",True,True


## Preprocessing

The dataset contains expressions to capture which versions are affected, and in which versions a vulnerability was fixed. 
We'll parse these expressions to convert them to intervals, so we can manipulate them more easily. 
The notation used is close to the one of Packagist, so we'll use our Packagist parser.

In [12]:
import sys

sys.path.append('../data')

from parsers import parse_or_empty, PackagistParser
from version import Version

parser = PackagistParser()

intervals = dict()

for expr in df_vuln.affect.drop_duplicates():
    intervals[expr] = parse_or_empty(parser, expr)
    
for expr in df_vuln.fix.drop_duplicates():
    intervals[expr] = parse_or_empty(parser, expr)

How many expressions did we successfully convert?

In [13]:
print('expressions:', len(intervals))
print('converted to non-empty:', len([k for k,v in intervals.items() if not v.empty]))
print('proportion:', len([k for k,v in intervals.items() if not v.empty]) / len(intervals))

print()

print('converted to empty:', len([k for k,v in intervals.items() if v.empty]))
print('\n'.join([k for k,v in intervals.items() if v.empty]))

expressions: 1311
converted to non-empty: 1301
proportion: 0.992372234935164

converted to empty: 10
>=3.0.0-alpha.1 <3.0.0
>=10.0.0-alpha.0 <10.0.0-beta.1
>=3.0.0-rc1 <3.0.0
>=2.0.0-alpha <2.0.0-alpha8
<0.0.0
>=4.0.0-alpha, <4.0.0-beta.2
>=5.2.2, <5.2.2.1
>=4.2.11, <4.2.11.1||>=5.0.7, <5.0.7.2||>=5.1.6, <5.1.6.2||>=5.2.2, <5.2.2.1
< 1.1.rc
1.1.rc


We label each release as being "affected" or not.

In [14]:
df_affected = (
    df_required
    .merge(
        df_vuln[['id', 'ecosystem', 'package', 'affect', 'fix']],
        how='inner',
        on=['ecosystem', 'package'],
    )
    # Ignore those for which we cannot deduce a fixed or affected release
    [lambda d: ~d['affect'].isin([k for k,v in intervals.items() if v.empty])]
    [lambda d: ~d['fix'].isin([k for k,v in intervals.items() if v.empty])]
    # Tag releases as fixed or affected
    .assign(affected=lambda d: 
        d.apply(axis=1, func=lambda s: 
            Version.from_string(s.version) in intervals[s.affect],
        )
    )
    .assign(fixed=lambda d: 
        d.apply(axis=1, func=lambda s: 
            Version.from_string(s.version) in intervals[s.fix],
        )
    )
)

In [15]:
(
    df_affected
    .groupby(['ecosystem'])
    .agg({
        'id': 'nunique', 
        'package': 'nunique',
        'version': 'count',
        'affected': 'sum',
    })
)

,id,package,version,affected
ecosystem,,,,
NPM,609,315,59521,33513
Rubygems,308,96,34311,14441


## Affected major branches

Our focus being on backporting updates in previous major branches, let's annotate, for each major of each package, whether it was affected and fixed.
We remove from our dataset all packages for which we do not have any affected or fixed release. 

In [16]:
df_major = (
    df_affected
    .groupby(['ecosystem', 'id', 'package', 'major'], as_index=False)
    .agg(
        affected=('affected', 'max'), 
        has_fix=('fixed', 'max'), 
        affect=('affect', 'first'),
        minrank=('rank', 'min'),
        maxrank=('rank', 'max'),
    )
    .eval('fixed = affected and has_fix')
    .eval('notfixed = affected and not has_fix')
    
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .filter(lambda g: g.has_fix.max() & g.affected.max())
)

In [17]:
(
    df_major
    .groupby(['ecosystem'])
    .agg({
        'id': 'nunique', 
        'package': 'nunique',
        'major': 'count',
        'affected': 'sum',
        'fixed': 'sum',
        'has_fix': 'sum',
        'notfixed': 'sum',
    })
)

,id,package,major,affected,fixed,has_fix,notfixed
ecosystem,,,,,,,
NPM,541,292,2390,1376,536,591,840
Rubygems,293,91,1225,650,330,357,320


We distinguish between the three following situations: 

 - A major is affected by a vulnerability, and is not fixed (affected = True, fixed = False);
 - A major is affected by a vulnerability and has a fix deployed (affected = True, fixed = True);
 - A major is affected by a vulnerability, and a fix is deployed as next major (next major has affected = False, and has_fix = True).
 
So in the above table, the difference between "fixed" and "has_fix" corresponds to the number of vulnerabilities having been fixed in a major release.

The number of backported fixes can be deduced from this dataframe: any vulnerability being fixed in at least two distinct major branches imply at least one backport (the number of major branches being the number of backport + 1). 

In [18]:
(
    df_major
    .groupby(['ecosystem', 'package', 'id'], as_index=False)
    .agg(
        number_of_fixes=('has_fix', 'sum'),
        number_of_fixed=('fixed', 'sum'),
        number_of_affected=('affected', 'sum'),
    )
    .eval("""
    fixed_by_next_major = number_of_fixes == 1 and number_of_fixed == 0
    fixed_in_current_major = number_of_fixes == 1 and number_of_fixed == 1
    fixed_with_backport = number_of_fixes > 1
    """)
    .assign(strategy=lambda d: d[['fixed_by_next_major', 'fixed_in_current_major', 'fixed_with_backport']].idxmax(axis=1))
    .groupby(['ecosystem', 'strategy'])
    .agg(
        vulnerabilities=('id', 'nunique'),
        packages=('package', 'nunique'), 
    )
    .pipe(lambda df:
        pandas.concat([
            df, 
            df
            .groupby(['ecosystem'])
            .apply(lambda g: g / g.sum())           
            .rename(columns=lambda s: 'prop_{}'.format(s))
        ], axis=1)
    )
    .style
    .format('{:.1%}', subset=['prop_vulnerabilities', 'prop_packages',])
    .background_gradient(vmin=0, vmax=1, subset=['prop_vulnerabilities', 'prop_packages',])
)


Let's look at the number of fixed/affected branches per package:

In [19]:
(
    df_major
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .agg({
        'major': 'count',
        'affected': 'sum', 
        'has_fix': 'sum',
        'fixed': 'sum',
        'notfixed': 'sum',
    })
    .eval("""
    p_affected = affected / major
    p_fixed = fixed / affected
    p_notfixed = notfixed / affected
    """)
    .groupby('ecosystem')
    .describe()
    [['major', 'affected', 'fixed', 'has_fix', 'p_affected', 'p_fixed', 'p_notfixed']]
    .loc[:, (slice(None), ['mean', '50%'])]    
)

major       affected          fixed        has_fix       \
               mean  50%      mean  50%      mean  50%      mean  50%   
ecosystem                                                               
NPM        4.417745  3.0  2.543438  2.0  0.990758  1.0  1.092421  1.0   
Rubygems   4.180887  4.0  2.218430  2.0  1.126280  1.0  1.218430  1.0   

          p_affected             p_fixed      p_notfixed       
                mean       50%      mean  50%       mean  50%  
ecosystem                                                      
NPM         0.722615  0.833333  0.577476  0.5   0.422524  0.5  
Rubygems    0.620795  0.571429  0.634585  0.5   0.365415  0.5

This shows that on average, from 57% to 83% of the major branches are affected by the vulnerability. 
A fix is deployed in 37% to 42% of the affected major branches (leaving the other major branches affected).

Looking at median values, a vulnerability affects 2 major branches (out of 3 or 4), and is fixed in 1 major branch, leaving 1 major branch affected.

Let's have a look at the proportion of vulnerabilities in function of the number of affected and fixed major branches.

In [20]:
(
    df_major
    .groupby(['ecosystem', 'id', 'package'], as_index=False)
    .agg({
        'affected': 'sum', 
        'fixed': 'sum',
    })
    .assign(fixed=lambda d: d.fixed.where(d.fixed <= 4, '5+'))
    .assign(affected=lambda d: d.affected.where(d.affected <= 4, '5+'))
    .groupby(['fixed', 'affected'])
    .id
    .count()
    .unstack()
    .pipe(lambda df: df / df.sum().sum())
    .assign(total=lambda d: d.sum(axis=1))
    .style
    .format('{:.2%}')
    .background_gradient(axis=None)
)

affected,1,2,3,4,5+,total
fixed,,,,,,
0,2.88%,1.80%,0.84%,0.36%,1.80%,7.67%
1,35.25%,22.18%,9.23%,7.43%,8.27%,82.37%
2,nan%,3.00%,2.88%,1.68%,1.56%,9.11%
3,nan%,nan%,0.12%,nan%,0.36%,0.48%
4,nan%,nan%,nan%,nan%,0.12%,0.12%
5+,nan%,nan%,nan%,nan%,0.24%,0.24%


Around 7.6% of the vulnerabilities are only fixed in a new major release (i.e., no backport).

Most of the vulnerabilities are fixed in at least one of the affected major branch (82%). This is not surprising when there is only one major branch being affected (35%) but should be considered as unsafe when multiple major are subject to the vulnerability (65%). 

## Affected dependent packages

It would be interesting to see how many dependent packages are still affected by each vulnerability. We expect this number to be lower for packages having backported a security fix. 
However, some vulnerabilities are old(er), and some of them affect (very) old versions, implying that any comparison would be biased. Moreover, some dependent packages could have been abandoned, hence not adopting a newer version (fixed or not) anyway. Finally, we only have the dependencies for the latest snapshot, hence we will simply count the (absolute) number of dependent packages that: 

 - are still relying on an older major branch affected by the vulnerability, and not fixed;
 - are still relying on an older major branch affected by the vulnerability, and fixed (i.e., they benefit from the backported fix).

In [21]:
df_aff_dep = (
    df_major
    # Get latest major
    .merge(
        df_major
        .groupby(['ecosystem', 'package'], as_index=False)
        .agg(latest_major=('major', 'max')),
        how='inner',
        on=['ecosystem', 'package'],
    )
    # Get latest fix to identify backports
    .merge(
        df_major
        .query('has_fix')
        .groupby(['ecosystem', 'package'], as_index=False)
        .agg(latest_fixed_major=('major', 'max')),
        how='inner', 
        on=['ecosystem', 'package'],
    )
    # Merge dependents
    .merge(
        df_dependents[['ecosystem', 'source', 'target', 'selected']],
        how='inner',
        left_on=['ecosystem', 'package'],
        right_on=['ecosystem', 'target'],
    )
    # Keep selected major
    .query('minrank <= selected <= maxrank')
    .eval('backported = fixed and latest_fixed_major > major')
)

### How many packages depend on the vulnerable ones?

In [22]:
(
    df_aff_dep
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,541,292,108008,513714
Rubygems,293,91,7234,74786


### How many packages depend on an affected major branch? 

Remember that it is not meaningful to look at these numbers proportionally to the above ones!

In [23]:
(
    df_aff_dep
    .query('affected')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,493,271,93725,386005
Rubygems,222,73,3677,21271


### How many of these packages depend on a previous (affected) major branch? 

Remember that it is not meaningful to look at these numbers proportionally to the above ones!

In [24]:
(
    df_aff_dep
    .query('affected and major < latest_major')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    ) 
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,329,177,12663,28335
Rubygems,150,43,616,2131


### How many dependent packages benefit from a backport?

These numbers can be compared with the above ones. 

In [25]:
(
    df_aff_dep
    .query('affected and fixed and backported')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    )
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,87,35,2207,6464
Rubygems,97,15,196,1015


### How many dependent packages would benefit from a backport? 

These numbers can be compared with the ones above the above ones. 

In [26]:
(
    df_aff_dep
    .query('affected and not fixed')
    .groupby('ecosystem')
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        dependencies=('source', 'count')
    )
)

,vulnerabilities,required,source,dependencies
ecosystem,,,,
NPM,205,117,4884,8190
Rubygems,30,19,172,235


Putting everything together: 

In [27]:
(
    df_aff_dep
    .eval('on_latest_major = major == latest_major')
    [['ecosystem', 'id', 'package', 'source', 'backported', 'affected', 'fixed', 'on_latest_major']]
    .groupby(['ecosystem', 'on_latest_major', 'affected', 'fixed', 'backported'])
    .agg(
        vulnerabilities=('id', 'nunique'),
        required=('package', 'nunique'), 
        source=('source', 'nunique'),
        cases=('source', 'count')
    )
)

vulnerabilities  \
ecosystem on_latest_major affected fixed backported                    
NPM       False           False    False False                   165   
                          True     False False                   205   
                                   True  False                   107   
                                         True                     87   
          True            False    False False                   263   
                          True     True  False                   276   
Rubygems  False           False    False False                   130   
                          True     False False                    29   
                                   True  False                    33   
                                         True                     97   
          True            False    False False                   204   
                          True     False False                     1   
                                   True  False                    88   

                                                     required  source   cases  
ecosystem on_latest_major affected fixed backported                            
NPM       False           False    False False             87   14221   36274  
                          True     False False            117    4884    8190  
                                   True  False             84    7471   13681  
                                         True              35    2207    6464  
          True            False    False False            149   40819   91435  
                          True     True  False            163   88109  357670  
Rubygems  False           False    False False             27    1242   11806  
                          True     False False             18     166     228  
                                   True  False             21     417     888  
                                         True              15     196    1015  
          True            False    False False             60    4712   41709  
                          True     False False              1       7       7  
                                   True  False             43    3203   19133